In [24]:
import warnings; warnings.filterwarnings('ignore')
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
from keras.applications import Xception, VGG19, InceptionV3, imagenet_utils
import keras.backend as K
from umap import UMAP
import numpy as np
import glob, os

image_sizes = [32, 128]
vectorizers = ['Xception']
projectors = [UMAP(min_dist=0.0001, n_neighbors=20)]

class Projector:
  def __init__(self, *args, **kwargs):
    self.clf = kwargs.get('clf')
    
  def fit_transform(self, vecs):
    return self.clf.fit_transform(vecs)

class Image:
  def __init__(self, *args, **kwargs):
    self.path = kwargs.get('path')
    self.original = load_img(self.path)
    self.resized = {s: self.resize(s) for s in image_sizes}
    self.vectors = {v: self.vectorize(v) for v in vectorizers}
    
  def resize(self, n):
    '''
    Resize self.original into a square image with shape n,n while maintaining proportion
    '''
    s = self.original.size
    size = (n, int(n * s[1]/s[0])) if s[0] > s[1] else (int(n * s[0]/s[1]), n)
    return self.original.resize(size)
    
  def vectorize(self, vectorizer_name):
    '''
    Given the name of a vectorizer model, return a vector representation of this image
    '''
    if vectorizer_name == 'Xception':
      # VGG16, VGG19, and ResNet take 224×224 images; InceptionV3 and Xception take 299×299 inputs
      img = self.original.resize((299,299))
      arr = img_to_array(img)
      # only Xception requires preprocessing
      arr = imagenet_utils.preprocess_input(arr)
      # input shape must be n_images, h, w, colors: https://keras.io/preprocessing/image/
      arr = np.expand_dims(arr, axis=0)
      # extract the ith layer from the model (here, the -1th layer, or final layer)
      out = K.function([model.input], [model.layers[-1].output])([arr])
      # return a 1000, dim vector
      return out[0][0]
    else:
      raise Exception('Requested model not implemented', model_name)
    
# process each image, project each
images = [Image(path=i) for i in glob.glob('images/*')]
for i in vectorizers:
  vecs = np.array([k.vectors[i] for k in images])
  for j in projectors: 
    projection = Projector(clf=j).fit_transform(vecs)

In [25]:
projection

array([[-3.993591 ,  0.5255954],
       [-4.1110463,  2.2405868],
       [-3.432333 ,  1.2341529],
       [-4.0282307,  1.5279508],
       [-5.1479616,  1.2633549],
       [-4.847983 ,  0.6344916]], dtype=float32)